In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.get_experiment_by_name("Default")

<Experiment: artifact_location='/mnt/c/Users/Dev/desktop/mlops_project/experiment_tracking/mlruns/0', creation_time=1705920879772, experiment_id='0', last_update_time=1705920879772, lifecycle_stage='active', name='Default', tags={}>

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [10]:
for run in runs:
    print(f"run_id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f},accuracy: {run.data.metrics['accuracy']:.4f},model: {run.data.tags['model']}")

run_id: 4ccc503e819a44c5b5ce862c421a6ab6, rmse: 0.0569,accuracy: 0.9958,model: ExtraTreesClassifier
run_id: deb4d19603c747438a006c497f117c2a, rmse: 0.0582,accuracy: 0.9956,model: RandomForestClassifier
run_id: 991d6c3b744247608d9a3401e9515e36, rmse: 0.0594,accuracy: 0.9954,model: xgboost
run_id: 972efc8246b347e8beb77b09ad439e4d, rmse: 0.0597,accuracy: 0.9953,model: xgboost
run_id: 9188c8c64da648f0937e8f22723deb3a, rmse: 0.0597,accuracy: 0.9953,model: xgboost


In [37]:
from sklearn.metrics import mean_squared_error,r2_score
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import mlflow

def read_dafaframe(filename):
    df = pd.read_csv(filename)

    le = LabelEncoder()
    df['type'] = le.fit_transform(df['type'])

    encoder = OneHotEncoder()
    encoded_types = encoder.fit_transform(df['type'].values.reshape(-1, 1))

    label = le.classes_.tolist()
    wine_types = pd.DataFrame(encoded_types.toarray(), columns=label)

    df = pd.concat([df, wine_types], axis=1)

    for col, value in df.items():
        if col != 'type':
            df[col] = df[col].fillna(df[col].mean())

    return df


def process(df):
    df_dict =  df.drop(columns=['type','quality'])
    X = df_dict

    # target = 'quality'
    # y = df[target].values

    return X


In [13]:
run_id = '4ccc503e819a44c5b5ce862c421a6ab6'

df = read_dafaframe("wine_data/test_wine_data.csv")

In [14]:
client.download_artifacts(run_id=run_id, path='model', dst_path='.')

/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/mnt/c/Users/Dev/desktop/mlops_project/experiment_tracking/model'

In [16]:
X_test = process(df)

In [17]:
target = 'quality'
y_test = df[target].values

In [36]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1706190964573, description='Model Registry for Wine Quality\n', last_updated_timestamp=1706198868264, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1706190967102, current_stage='Production', description='', last_updated_timestamp=1706198868264, name='Wine-Quality-Register', run_id='4ccc503e819a44c5b5ce862c421a6ab6', run_link='', source='/mnt/c/Users/Dev/desktop/mlops_project/experiment_tracking/mlruns/1/4ccc503e819a44c5b5ce862c421a6ab6/artifacts/model', status='READY', status_message=None, tags={'Model': 'ExtraTreesClassifier'}, user_id=None, version=1>,
  <ModelVersion: aliases=[], creation_timestamp=1706191013431, current_stage='Staging', description='', last_updated_timestamp=1706191824623, name='Wine-Quality-Register', run_id='972efc8246b347e8beb77b09ad439e4d', run_link='', source='/mnt/c/Users/Dev/desktop/mlops_project/experiment_tracking/mlruns/1/972efc8246b347e8beb77b09ad439e4d/artifacts/models_mlflow', status='R